In [1]:
# Importing Qiskit
# from qiskit import *
# from qiskit.qasm2 import dumps
# from qiskit_aer import Aer

import numpy as np
# import operation as op
from functools import reduce
import random
from random import randrange

In [2]:
def hamming(bits, order):
    '''
    Takes a string of bits to be corrected (bob bits). Bit-wise sums the indices of elements which are '1'. The 0th bit stores the parity
    of the entire block. The location of the error is returned. 
    If the location is not '0', the current 0th parity is matched with that of the parity obtained after flipping the bit at the location obtained. 
    If the parity matches, then the error is found and corrected. If the parity doesn't match then there are more than 1 error.

    If the location is '0', then no error is present.
    
    '''
    
    loc = reduce(lambda x, y : x^y, [i for i, bit in enumerate(bits) if bit == 1])    # x^y will apply xor to the binary rep of i -> index of 1s
    # loc = reduce(op.XOR, [i for i, bit in enumerate(bob_bits) if bit == 1])
    
    binary_rep = f"{bin_rep(loc, order)}"

    par = sum(bits[i] for i in range(0, len(bits)))%2

    if loc != 0 :
        if par != 0 :
            err_count = 1
            print(f"Error found at location : {loc}")

        else :
            err_count = 2
            print("More than 1 errors found")
            
    else : 
        err_count = 0
        print("No errors found")

    
    return err_count, loc, binary_rep

In [3]:
def encode_parity(alice_bits, order):
    # order = np.ceil(np.log2(len(alice_bits)))
    PARITY_DICT, bin_parity = parity(order)
    # print_extras(choice = ['2', '1'])
    
    sub_block = int(2**(order - 1))
    parity_of = np.zeros((len(PARITY_DICT), sub_block)).astype(int)   # An array to store the locations affecting the parity p
    
    for p in range(1, order+1) :    # checking for 1 at position p. eg : bin(45) = 101101
    
        bit_index = 2**(p-1)
        highlight = np.zeros(2**order).astype(int)                        # Highlights the locations affected by the current parity bit
        # print(f"bin rep of {bit_index = } : {bin_parity[p]}")
        
        for i in range(sub_block):                                         #  Order-1 = 5. range(5) = 0, 1, 2, 3, 4 => order-2
            bin_index = bin_rep(i, order-1)                                # Index(in binary formin binary form) for the data bits : 5 digits : 00010
            bin_index = bin_index[: order-p] + '1' + bin_index[order-p :]
            index = int(bin_index, base = 2)                                # Gives the index(int) of the elements to be considered for the current parity element
            
            parity_of[p, i] = index
            highlight[index] = 1
    
        PARITY_DICT[bit_index] = np.mod( sum( alice_bits[parity_of[p, i]] for i in range(sub_block) if bit_index != parity_of[p, i] ), 2 )
    
        if PARITY_DICT[bit_index] != alice_bits[bit_index] : 
            alice_bits[bit_index] = np.mod(alice_bits[bit_index] + 1, 2)
            
        # print_extras('3')    # print(highlight.reshape(dim, dim))
    
    PARITY_DICT[0] = sum( alice_bits[i] for i in range(1, 2**order) )%2
    alice_bits[0] = PARITY_DICT[0]
    # print(f"Parity locations : \n{parity_of[1:]}")
    
    print("\n Hamming Results : ", hamming(alice_bits, order))
    print(f" Alice uncorrected {block(alice_bits, order)}")
    
    return alice_bits, PARITY_DICT

In [4]:
# Block is the representation of an array consisting the binary data string that has been reshaped as a square matrix
# It has dimensions : dim*dim, where dim = 2**(order/2)

def bin_rep(loc, order):
    '''
    Takes a number(int) and order/precision(int) as an input, and returns the binary form with the requested precision.
    '''
    bin_loc = bin(loc)[2:]
    bin_rep = f"{'0'*(order - len(bin_loc))}{bin_loc}"
    
    return bin_rep


def parity(order):
    '''
    Takes in order(int) as a parameter. Returns 2 arrays : 
       - parity_bits : an array containing '0' and the powers of 2 till 2^(order-1)
       - bin_parity : an array of the binary representation of elements of parity_bits   
    '''
    PARITY_DICT = {0:0, **{2**i : 0 for i in range(order)}}
    # parity_bits = np.array([0] + [2**i for i in range(order)]).astype(int)
    bin_parity = np.array([bin_rep(int(i), int(order)) for i in PARITY_DICT.keys()])

    return PARITY_DICT, bin_parity
    
    
def parity_locs(order):
    '''
    Takes in order(int) as a parameter. Returns an array :
        - parity_locs : A block(array reshaped as square matrix) with 1 at the locations of parity bits
    '''
    parity_locs = np.full(2**order, '-', dtype = object)
    PARITY_DICT = parity(order)[0]
    
    for loc in PARITY_DICT.keys() : parity_locs[loc] = '1'

    return parity_locs


# '''
# def print_extras(choice = 0):

#     if choice == 0 : choice = [i in range(4)] 
    
#     if '1' in choice :
#         print(f"Parity Dictionary : {PARITY_DICT} \nBinary representation of parity bit indices : {bin_parity}")

#     if '2' in choice :
#         print(f"\nShape of the block : {dim}*{dim}")
#         print("Uncorrected Alice block : \n", alice_bits.reshape(dim, dim))
#         print(f'Parity bit locations : \n{parity_locs(order).reshape(dim, dim)}')

#     if '3' in choice : 
#         print(highlight.reshape(dim, dim))
# '''

In [5]:
def block(bits, order):
    dim = int(2**(order/2))

    if not order%2 : 
        print(f"")
        return(f"block : \n {bits.reshape(dim, dim)} \n Shape of the block : {dim}*{dim}")

    else :
        return(f"bit string(Order is odd, can't project to a block) : \n {bits} \n Shape of the block : {bits.shape}")

In [6]:
order = 6
dim = int(2**(order/2))
alice_bits = np.array([random.randint(0, 1) for _ in range(2**order)])    # Randomly fills the array with 0/1
alice_bits[2**order - 1] = 0
print(f" Alice uncorrected {block(alice_bits, order)}")


 Alice uncorrected block : 
 [[0 0 0 0 0 0 0 1]
 [1 0 0 0 1 1 1 0]
 [1 0 0 0 1 1 1 1]
 [1 1 0 1 0 0 1 0]
 [1 1 0 1 0 1 1 0]
 [0 0 1 1 0 0 1 0]
 [0 1 1 0 1 1 1 1]
 [1 1 0 1 1 1 1 0]] 
 Shape of the block : 8*8


In [7]:
alice_bits, PARITY_DICT = encode_parity(alice_bits, order)
print(f"\n Parity Dictionary : {PARITY_DICT} \n Alice (parity embedded) {block(alice_bits, order)}")

No errors found

 Hamming Results :  (0, 0, '000000')

 Alice uncorrected block : 
 [[1 0 0 0 1 0 0 1]
 [0 0 0 0 1 1 1 0]
 [0 0 0 0 1 1 1 1]
 [1 1 0 1 0 0 1 0]
 [1 1 0 1 0 1 1 0]
 [0 0 1 1 0 0 1 0]
 [0 1 1 0 1 1 1 1]
 [1 1 0 1 1 1 1 0]] 
 Shape of the block : 8*8


 Parity Dictionary : {0: 1, 1: 0, 2: 0, 4: 1, 8: 0, 16: 0, 32: 1} 
 Alice (parity embedded) block : 
 [[1 0 0 0 1 0 0 1]
 [0 0 0 0 1 1 1 0]
 [0 0 0 0 1 1 1 1]
 [1 1 0 1 0 0 1 0]
 [1 1 0 1 0 1 1 0]
 [0 0 1 1 0 0 1 0]
 [0 1 1 0 1 1 1 1]
 [1 1 0 1 1 1 1 0]] 
 Shape of the block : 8*8


In [8]:
bob_bits = np.copy(alice_bits)

rounds = 0
accuracy = 0
num_errors = 1    # To decide for simulation : could be 0, 1, 2, or more
# pos = [random.randrange(0, 16) for _ in range(num_errors)]
pos = 5    # For simulation purposes, keep it fixed. Else use the above commented statement
print(f"{num_errors=}, {pos=}")
# pos = [5, 7]

rounds += 1

bob_bits[pos] = np.mod(bob_bits[pos]+1, 2)

print(f"\nAlice {block(alice_bits, order)} \n\nBob {block(bob_bits, order)}")

num_errors=1, pos=5



Alice block : 
 [[1 0 0 0 1 0 0 1]
 [0 0 0 0 1 1 1 0]
 [0 0 0 0 1 1 1 1]
 [1 1 0 1 0 0 1 0]
 [1 1 0 1 0 1 1 0]
 [0 0 1 1 0 0 1 0]
 [0 1 1 0 1 1 1 1]
 [1 1 0 1 1 1 1 0]] 
 Shape of the block : 8*8 

Bob block : 
 [[1 0 0 0 1 1 0 1]
 [0 0 0 0 1 1 1 0]
 [0 0 0 0 1 1 1 1]
 [1 1 0 1 0 0 1 0]
 [1 1 0 1 0 1 1 0]
 [0 0 1 1 0 0 1 0]
 [0 1 1 0 1 1 1 1]
 [1 1 0 1 1 1 1 0]] 
 Shape of the block : 8*8


In [9]:
out = hamming(bob_bits, order)
print("Error counts : ", out[0], ", loc : ", out[1:])

if num_errors == out[0] and pos == out[1]:
    if num_errors == 1:
        bob_bits[out[1]] = np.mod(bob_bits[out[1]] + 1, 2)
        
    accuracy += 1

print(f"\n {num_errors = }, {type(num_errors) = }, {out[0] = }, {type(out[0]) = }")
print(f"\n {pos = }, {type(pos) = }, {out[1] = }, {type(out[1]) = }")

accuracy /= rounds
# type(out[0]), print(out[0])
print(f"\nAlice {block(alice_bits, order)} \n\nBob {block(bob_bits, order)}")
print(f"\n {accuracy = }")
if bob_bits[pos] == alice_bits[pos] : print(f"\n Bob bits and alice bits are same")
# print(f'{out[0]:04b}') 

Error found at location : 5
Error counts :  1 , loc :  (5, '000101')

 num_errors = 1, type(num_errors) = <class 'int'>, out[0] = 1, type(out[0]) = <class 'int'>

 pos = 5, type(pos) = <class 'int'>, out[1] = 5, type(out[1]) = <class 'int'>



Alice block : 
 [[1 0 0 0 1 0 0 1]
 [0 0 0 0 1 1 1 0]
 [0 0 0 0 1 1 1 1]
 [1 1 0 1 0 0 1 0]
 [1 1 0 1 0 1 1 0]
 [0 0 1 1 0 0 1 0]
 [0 1 1 0 1 1 1 1]
 [1 1 0 1 1 1 1 0]] 
 Shape of the block : 8*8 

Bob block : 
 [[1 0 0 0 1 0 0 1]
 [0 0 0 0 1 1 1 0]
 [0 0 0 0 1 1 1 1]
 [1 1 0 1 0 0 1 0]
 [1 1 0 1 0 1 1 0]
 [0 0 1 1 0 0 1 0]
 [0 1 1 0 1 1 1 1]
 [1 1 0 1 1 1 1 0]] 
 Shape of the block : 8*8

 accuracy = 1.0

 Bob bits and alice bits are same
